In [81]:
# importing necessary libraries
import pandas as pd
import yfinance as yf
from datetime import timedelta
import numpy as np

In [82]:
spy_stocklist = pd.read_csv(r'raw_data/sp_500_historical_components.csv')

# Data cleaning
In this portion of the code we will 
1) Get the list of stocks that ever existed from sp_500_historical_components.csv from https://github.com/hanshof/sp500_constituents/blob/main/sp_500_historical_components.csv
2) Use yfinance library to get historical data (2012-2020) OHLCV, PE ratio, PB ratio from the lists of stocks, indicate their presence in the stock data at any point of time on a daily level. we get 2012 data for a buffer to calculate moving averages later on
3) Filter out stocks that newly entered the stock market before 2016, this is to ensure that we have sufficient training data for each stock

In [83]:

# Convert 'date' column to datetime
spy_stocklist['date'] = pd.to_datetime(spy_stocklist['date'])

# Ensure 'tickers' is string
spy_stocklist['tickers'] = spy_stocklist['tickers'].astype(str)
spy_stocklist

,date,tickers
0,1996-01-02,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1,1996-01-03,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
2,1996-01-04,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
3,1996-01-10,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
4,1996-01-11,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
...,...,...
3477,2025-08-19,"A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP..."
3478,2025-08-20,"A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP..."
3479,2025-08-21,"A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP..."
3480,2025-08-22,"A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP..."


In [84]:
spy_stocklist_filtered = spy_stocklist[(spy_stocklist['date'] >= '2013-01-01') & (spy_stocklist['date'] <= '2021-01-08')]

spy_stocklist_filtered

,date,tickers
1908,2013-01-02,"A,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,..."
1909,2013-01-03,"A,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,..."
1910,2013-01-08,"A,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,..."
1911,2013-01-14,"A,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,..."
1912,2013-01-15,"A,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,..."
...,...,...
2625,2020-10-07,"A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,..."
2626,2020-10-12,"A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,..."
2627,2020-11-17,"A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,..."
2628,2020-12-21,"A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,..."


In [85]:
# Split the tickers into lists
spy_stocklist_filtered_expanded_unexploded = (
    spy_stocklist_filtered.assign(
        ticker=spy_stocklist_filtered["tickers"].str.split(",")
    )
    .drop(columns="tickers")
    .reset_index(drop=True)
)

# Apply mapping to each list of tickers
updated_ticker_mapping = {
    'GOOGL': 'GOOG',
    'FB': 'META',
    'TWTR': 'X',
}

spy_stocklist_filtered_expanded_unexploded["ticker"] = (
    spy_stocklist_filtered_expanded_unexploded["ticker"]
    .apply(lambda lst: [updated_ticker_mapping.get(t, t) for t in lst])
)


# Explode the lists into separate rows
spy_stocklist_filtered_expanded = (
    spy_stocklist_filtered_expanded_unexploded
    .explode("ticker")
    .reset_index(drop=True)
)

spy_stocklist_filtered_expanded 


,date,ticker
0,2013-01-02,A
1,2013-01-02,AAPL
2,2013-01-02,ABBV
3,2013-01-02,ABC
4,2013-01-02,ABT
...,...,...
344472,2021-01-07,YUM
344473,2021-01-07,ZBH
344474,2021-01-07,ZBRA
344475,2021-01-07,ZION


In [86]:
def get_monthly_survivors(df):
    df['year_month'] = df['date'].dt.to_period('M')
    
    # Get first date of each month
    month_starts = df.groupby('year_month')['date'].min().reset_index()
    month_starts.columns = ['year_month', 'start_date']
    
    results = []
    
    for i in range(len(month_starts) - 1):
        start = month_starts.loc[i, 'start_date']
        end = month_starts.loc[i + 1, 'start_date']

        # Filter month data
        month_df = df[(df['date'] >= start) & (df['date'] < end)]
        if month_df.empty:
            continue

        # All tickers ever observed in this month
        all_tickers = set().union(*month_df['ticker'])

        # Intersection across all days → surviving tickers
        surviving = set(month_df.iloc[0]['ticker'])
        for tickers in month_df['ticker']:
            surviving &= set(tickers)

        results.append({
            'date': start,
            'ticker_list': sorted(list(surviving)),
        })

    return pd.DataFrame(results)

survivors_df = get_monthly_survivors(spy_stocklist_filtered_expanded_unexploded)

# Convert 'date' to first day of the month and create 'month' column
survivors_df['month'] = survivors_df['date'].dt.to_period('M').dt.to_timestamp()
# Set 'month' as index
survivors_df.set_index('month', inplace=True)
# Create full month range
full_month_range = pd.date_range(start='2013-01-01', end='2020-12-31', freq='MS')
# Reindex to include all months 
survivors_df = survivors_df.reindex(full_month_range)

# Forward-fill missing months
survivors_df.ffill(inplace=True)
survivors_df = survivors_df.reset_index()
survivors_df.rename(columns={'index': 'month'}, inplace=True)

survivors_df

,month,date,ticker_list
0,2013-01-01,2013-01-02,"[A, AAPL, ABBV, ABC, ABT, ACN, ADBE, ADI, ADM,..."
1,2013-02-01,2013-02-04,"[A, AAPL, ABBV, ABC, ABT, ACN, ADBE, ADI, ADM,..."
2,2013-03-01,2013-03-11,"[A, AAPL, ABBV, ABC, ABT, ACN, ADBE, ADI, ADM,..."
3,2013-04-01,2013-04-01,"[A, AAPL, ABBV, ABC, ABT, ACN, ADBE, ADI, ADM,..."
4,2013-05-01,2013-05-01,"[A, AAPL, ABBV, ABC, ABT, ACN, ADBE, ADI, ADM,..."
...,...,...,...
91,2020-08-01,2020-06-22,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
92,2020-09-01,2020-09-18,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
93,2020-10-01,2020-10-07,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
94,2020-11-01,2020-11-17,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."


In [87]:
### getting list of unique tickers that ever entered the SP500 between 2013-2020

spy_stocklist_final = spy_stocklist_filtered_expanded[
    (spy_stocklist_filtered_expanded['date'] >= '2013-01-01') &
    (spy_stocklist_filtered_expanded['date'] <= '2020-12-31')]

unique_tickers = spy_stocklist_final['ticker'].unique().tolist()
print(f"Total unique tickers in S&P 500 from 2013 to 2020: {len(unique_tickers)}")
unique_tickers

Total unique tickers in S&P 500 from 2013 to 2020: 639


['A',
 'AAPL',
 'ABBV',
 'ABC',
 'ABT',
 'ACN',
 'ADBE',
 'ADI',
 'ADM',
 'ADP',
 'ADSK',
 'AEE',
 'AEP',
 'AES',
 'AET',
 'AFL',
 'AIG',
 'AIV',
 'AIZ',
 'AKAM',
 'ALL',
 'ALXN',
 'AMAT',
 'AMD',
 'AMGN',
 'AMP',
 'AMT',
 'AMZN',
 'AN',
 'ANDV',
 'ANF',
 'ANTM',
 'AON',
 'APA',
 'APC',
 'APD',
 'APH',
 'APOL',
 'APTV',
 'ARG',
 'ATI',
 'AVB',
 'AVP',
 'AVY',
 'AXP',
 'AZO',
 'BA',
 'BAC',
 'BAX',
 'BBBY',
 'BBT',
 'BBY',
 'BCR',
 'BDX',
 'BEN',
 'BF-B',
 'BIG',
 'BIIB',
 'BK',
 'BKNG',
 'BLK',
 'BLL',
 'BMS',
 'BMY',
 'BRCM',
 'BRK-B',
 'BSX',
 'BTUUQ',
 'BWA',
 'BXP',
 'C',
 'CA',
 'CAG',
 'CAH',
 'CAM',
 'CAT',
 'CB',
 'CBRE',
 'CBS',
 'CCI',
 'CCL',
 'CELG',
 'CERN',
 'CF',
 'CFN',
 'CHRW',
 'CI',
 'CINF',
 'CL',
 'CLF',
 'CLX',
 'CMA',
 'CMCSA',
 'CME',
 'CMG',
 'CMI',
 'CMS',
 'CNP',
 'CNX',
 'COF',
 'COG',
 'COL',
 'COP',
 'COST',
 'COV',
 'CPB',
 'CRM',
 'CSCO',
 'CSX',
 'CTAS',
 'CTL',
 'CTSH',
 'CTXS',
 'CVC',
 'CVS',
 'CVX',
 'D',
 'DD',
 'DE',
 'DF',
 'DFS',
 'DG',
 'DGX',


In [88]:
### now we will use the yfinance library to get the historical data for these tickers that ever existed in the SP500 between 2013-2020
def get_stock_data(ticker, start_date, end_date):
    try:
        # Convert string to datetime so we can safely add 1 day, because yfinance end date is exclusive we add one day to include the end date
        end_date_dt = pd.to_datetime(end_date) + pd.Timedelta(days=1)
        
        stock = yf.Ticker(ticker)
        stock_data = stock.history(
            start=start_date,
            end=end_date_dt,
            auto_adjust=True,  # adjust for stock splits/dividends
            actions=True       # include Dividends and Stock Splits columns
        )
        
        if stock_data.empty:
            return pd.DataFrame()
        
        stock_data["ticker"] = ticker
        return stock_data.reset_index()

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.DataFrame()

In [89]:
start_date = '2012-01-01' # we start from 2012 to have a buffer for moving averages calculation
end_date = '2021-01-31' # we end at 2020-12-31, 2019-2020 will be held out for stress testing 
                        # 1 month additional buffer when we lead future expected returns predictino
yfinance_data = []
failed_tickers = []
successful_tickers = []
for ticker in unique_tickers:
    print(f"Fetching data for {ticker}...")
    ticker_data = get_stock_data(ticker, start_date, end_date)
    if not ticker_data.empty:
        yfinance_data.append(ticker_data)
        successful_tickers.append(ticker)
    else:
        failed_tickers.append(ticker)

    

Fetching data for A...
Fetching data for AAPL...
Fetching data for ABBV...
Fetching data for ABC...


$ABC: possibly delisted; no timezone found


Fetching data for ABT...
Fetching data for ACN...
Fetching data for ADBE...
Fetching data for ADI...
Fetching data for ADM...
Fetching data for ADP...
Fetching data for ADSK...
Fetching data for AEE...
Fetching data for AEP...
Fetching data for AES...
Fetching data for AET...
Fetching data for AFL...
Fetching data for AIG...
Fetching data for AIV...
Fetching data for AIZ...
Fetching data for AKAM...
Fetching data for ALL...
Fetching data for ALXN...


$ALXN: possibly delisted; no timezone found


Fetching data for AMAT...
Fetching data for AMD...
Fetching data for AMGN...
Fetching data for AMP...
Fetching data for AMT...
Fetching data for AMZN...
Fetching data for AN...
Fetching data for ANDV...
Fetching data for ANF...
Fetching data for ANTM...


...

Fetching data for NOW...
Fetching data for VIAC...


$VIAC: possibly delisted; no timezone found


Fetching data for WRB...
Fetching data for ODFL...
Fetching data for TFC...
Fetching data for J...
Fetching data for LYV...
Fetching data for STE...
Fetching data for ZBRA...
Fetching data for PAYC...
Fetching data for TT...
Fetching data for CARR...
Fetching data for OTIS...
Fetching data for RTX...
Fetching data for HWM...
Fetching data for DPZ...
Fetching data for DXCM...
Fetching data for WST...
Fetching data for BIO...
Fetching data for TDY...
Fetching data for TYL...
Fetching data for LUMN...
Fetching data for CTLT...


$CTLT: possibly delisted; no timezone found


Fetching data for ETSY...
Fetching data for POOL...
Fetching data for VNT...
Fetching data for VTRS...
Fetching data for TSLA...


In [90]:
# combine all individual DataFrames into one big DataFrame
yfinance_df = pd.concat(yfinance_data, ignore_index=True)
yfinance_df["Date"] = pd.to_datetime(yfinance_df["Date"])

# preview result
yfinance_df.drop(columns=['Capital Gains'], inplace=True)
yfinance_df['Date'] = yfinance_df['Date'].dt.tz_localize(None)

yfinance_df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker
0,2012-01-03,22.776930,23.507775,22.713378,23.183662,4156394,0.0,0.0,A
1,2012-01-04,22.973936,23.107394,22.618046,22.999355,4651845,0.0,0.0,A
2,2012-01-05,22.802342,23.717486,22.700660,23.514122,6842651,0.0,0.0,A
3,2012-01-06,23.571322,23.870016,23.393379,23.768333,4711400,0.0,0.0,A
4,2012-01-09,23.908145,24.416559,23.812817,24.391138,4429563,0.0,0.0,A
...,...,...,...,...,...,...,...,...,...
1138630,2021-01-25,285.000000,300.133331,279.606659,293.600006,123520200,0.0,0.0,TSLA
1138631,2021-01-26,297.126678,298.633331,290.533325,294.363342,69394800,0.0,0.0,TSLA
1138632,2021-01-27,290.116669,297.166656,286.220001,288.053345,82002000,0.0,0.0,TSLA
1138633,2021-01-28,273.333344,282.666656,267.000000,278.476654,79134000,0.0,0.0,TSLA


In [91]:
# checking for stock split price adjustment, for example apple, check the dates around 2020-08-31 when apple had a 4-for-1 stock split
yfinance_df[yfinance_df['Date'].between('2020-08-24', '2020-09-07') & (yfinance_df['ticker'] == 'AAPL')]

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker
4458,2020-08-24,125.186335,125.271457,120.556204,122.423820,345937600,0.0,0.0,AAPL
4459,2020-08-25,121.295478,121.764813,119.695353,121.419495,211495600,0.0,0.0,AAPL
4460,2020-08-26,122.737524,123.527857,121.669963,123.070679,163022400,0.0,0.0,AAPL
4461,2020-08-27,123.673785,124.006940,120.454084,121.599464,155552400,0.0,0.0,AAPL
4462,2020-08-28,122.574593,122.992862,121.178745,121.402473,187630000,0.0,0.0,AAPL
4463,2020-08-31,124.099341,127.426034,122.562445,125.519501,225702700,0.0,4.0,AAPL
4464,2020-09-01,129.137999,131.122352,126.968843,130.519257,151948100,0.0,0.0,AAPL
4465,2020-09-02,133.836243,134.215602,123.535164,127.815117,200119000,0.0,0.0,AAPL
4466,2020-09-03,123.447617,125.324955,117.212493,117.582123,257599600,0.0,0.0,AAPL
4467,2020-09-04,116.794224,120.325187,107.864675,117.659943,332607200,0.0,0.0,AAPL


In [92]:
#Membership Indicator 

# Create 'month' column 
yfinance_df['month'] = yfinance_df['Date'].dt.to_period('M').dt.to_timestamp()

# Merge with survivors_df on 'month'
merged_df = yfinance_df.merge(
    survivors_df[['month', 'ticker_list']],
    on='month',
    how='left'
)

# Check if ticker is in ticker_list for that month, 1 if yes, 0 if no
merged_df['is_in_sp500'] = merged_df.apply(
    lambda row: int(row['ticker'] in row['ticker_list']) if isinstance(row['ticker_list'], (list, set)) else 0,
    axis=1
)



merged_df.drop(columns=['ticker_list', 'month'], inplace=True)


In [93]:
#just to check
merged_df[merged_df['ticker'] == 'TSLA'].tail(25)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,is_in_sp500
1138610,2020-12-23,210.733337,217.166672,207.523331,215.326660,99519000,0.0,0.0,TSLA,1
1138611,2020-12-24,214.330002,222.029999,213.666672,220.589996,68596800,0.0,0.0,TSLA,1
1138612,2020-12-28,224.836670,227.133331,220.266663,221.229996,96835800,0.0,0.0,TSLA,1
1138613,2020-12-29,220.333328,223.300003,218.333328,221.996674,68732400,0.0,0.0,TSLA,1
1138614,2020-12-30,224.000000,232.199997,222.786667,231.593338,128538000,0.0,0.0,TSLA,1
1138615,2020-12-31,233.330002,239.573334,230.373337,235.223328,148949700,0.0,0.0,TSLA,1
1138616,2021-01-04,239.820007,248.163330,239.063339,243.256668,145914600,0.0,0.0,TSLA,0
1138617,2021-01-05,241.220001,246.946671,239.733337,245.036667,96735600,0.0,0.0,TSLA,0
1138618,2021-01-06,252.830002,258.000000,249.699997,251.993332,134100000,0.0,0.0,TSLA,0
1138619,2021-01-07,259.209991,272.329987,258.399994,272.013336,154496700,0.0,0.0,TSLA,0


In [94]:
# filtering stocks that entered the stock market after 2016-01-01 to ensure we have enough training data per stock

yfinance_df = merged_df
first_trade = yfinance_df.groupby("ticker")["Date"].min().reset_index()
first_trade["Date"] = pd.to_datetime(first_trade["Date"])
first_trade.columns = ["ticker", "first_trade_date"]

# keep only tickers that started before or on 2015-12-31
eligible_tickers = first_trade[first_trade["first_trade_date"] <= "2015-12-31"]["ticker"]

yfinance_df_filtered_relavant_stocks = filtered_df = yfinance_df[yfinance_df["ticker"].isin(eligible_tickers)].copy()


print(f"Total tickers after filtering: {eligible_tickers.nunique()}, tickers filtered out: {len(unique_tickers) - eligible_tickers.nunique()}")

Total tickers after filtering: 506, tickers filtered out: 133


In [95]:
# getting daily returns, monthly returns and adding to the dataframe
# we take 21 trading days as approximately 1 month

yfinance_df_filtered_relavant_stocks.sort_values(by=['ticker', 'Date'], inplace=True)

# Compute log returns within each ticker group
yfinance_df_filtered_relavant_stocks['daily_log_return'] = (
    yfinance_df_filtered_relavant_stocks
    .groupby('ticker')['Close']
    .transform(lambda x: np.log(x / x.shift(1)))
)

# Compute log volume change within each ticker group
yfinance_df_filtered_relavant_stocks['daily_volume_return'] = (
    yfinance_df_filtered_relavant_stocks
    .groupby('ticker')['Close']
    .transform(lambda x: np.log(x / x.shift(1)))
)

# Approx. 21 trading days ≈ 1 month
yfinance_df_filtered_relavant_stocks['monthly_log_return'] = (
    yfinance_df_filtered_relavant_stocks
    .groupby('ticker')['Close']
    .transform(lambda x: np.log(x / x.shift(21)))
)


## future volatility 
yfinance_df_filtered_relavant_stocks['monthly_log_return_future'] = (
    yfinance_df_filtered_relavant_stocks
    .groupby('ticker')['Close']
    .transform(lambda x: np.log(x.shift(-21) / x))
    
)

In [96]:
# getting monthly standard deviation (volatility) (TODO)
yfinance_df_filtered_relavant_stocks["monthly_var_3"] = (
    yfinance_df_filtered_relavant_stocks.groupby("ticker")["monthly_log_return"]
      .transform(lambda x: x.rolling(window=3, min_periods=3).var(ddof=1))
)

yfinance_df_filtered_relavant_stocks["daily_vol_21"] = (
    yfinance_df_filtered_relavant_stocks
    .groupby("ticker")["daily_log_return"]
    .transform(lambda x: x.rolling(window=21, min_periods=21).std(ddof=1))
)

### EDA for new stocks that entered after our 2013 cut off
- the problem with these stocks is that they might not have enough historical data to compute fields such as:
1) monthly returns
2) monthly variance (3 months)
3) EMA/SMA

In [97]:
start_date = pd.Timestamp("2013-01-01")

# ensure Date is Timestamp
yfinance_df_filtered_relavant_stocks["Date"] = pd.to_datetime(yfinance_df_filtered_relavant_stocks["Date"])

# find earliest date per ticker
first_dates = (
    yfinance_df_filtered_relavant_stocks
    .groupby("ticker")["Date"]
    .min()
    .reset_index(name="first_date")
)

# flag tickers that started trading after 2013-01-01
first_dates["entered_after_2013"] = (first_dates["first_date"] > start_date).astype(int)

# count and preview
num_after = first_dates["entered_after_2013"].sum()
num_total = len(first_dates)

print(f"{num_after} out of {num_total} tickers started trading after {start_date.date()} before 2015-12-31.")
print(first_dates.query("entered_after_2013 == 1").head())

25 out of 506 tickers started trading after 2013-01-01 before 2015-12-31.
   ticker first_date  entered_after_2013
4    ABBV 2013-01-02                   1
26   ALLE 2013-11-18                   1
38   ANET 2014-06-06                   1
88    CDW 2013-06-27                   1
91    CFG 2014-09-24                   1


### Creating new features
#### Moving averages
https://www.investopedia.com/ask/answers/122414/what-are-most-common-periods-used-creating-moving-average-ma-lines.asp we will use short:20 days, medium: 50 days, long: 100 days moving averages
1) Simple moving average SMA: SMA_20, SMA_50, SMA_100
2) Exponential Moving Average EMA https://www.investopedia.com/terms/e/ema.asp#toc-formula-for-exponential-moving-average-ema: EMA_20, EMA_50, EMA_100

#### RSI and MACD
1) Relative Strength Index 14 days https://www.investopedia.com/terms/r/rsi.asp : RSI
2) Moving Average Convergence/Divergence indicator: https://www.investopedia.com/terms/m/macd.asp: MACD_26, MACD_12, MACD_9 

#### Stock Splits
1) Forward stock split (Stock split > 1): Commonly known as a bullish indicator which provides more liquidity (when fractional shares werent common)
2) Reverse stock split (0 < Stock split < 1): Bearish indicator

In [98]:
def sma(series: pd.Series, window: int) -> pd.Series:
    """Simple Moving Average."""
    return series.rolling(window, min_periods=window).mean()

In [99]:
def ema(series: pd.Series, span: int) -> pd.Series:
    """Exponential Moving Average."""
    return series.ewm(span=span, adjust=False, min_periods=span).mean()

In [100]:
def rsi_wilder(series: pd.Series, period: int = 14) -> pd.Series:
    """Wilder's RSI (default 14)."""
    delta = series.diff()
    gain  = delta.clip(lower=0)
    loss  = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/period, adjust=False, min_periods=period).mean()
    avg_loss = loss.ewm(alpha=1/period, adjust=False, min_periods=period).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

In [101]:
def macd(series: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
    """
    MACD parts: returns DataFrame with MACD_Line, MACD_Signal, MACD_Hist.
    """
    ema_fast = ema(series, fast)
    ema_slow = ema(series, slow)
    macd_line = ema_fast - ema_slow
    macd_signal = macd_line.ewm(span=signal, adjust=False, min_periods=signal).mean()
    macd_hist = macd_line - macd_signal
    return pd.DataFrame(
        {"MACD_Line": macd_line, "MACD_Signal": macd_signal, "MACD_Hist": macd_hist},
        index=series.index
    )

In [102]:
temp_df = yfinance_df_filtered_relavant_stocks.copy()

# Ensure types/order; compute per-ticker
temp_df['Date'] = pd.to_datetime(temp_df['Date'])
temp_df = temp_df.drop_duplicates(subset=['ticker','Date']).sort_values(['ticker','Date'])


g = temp_df.groupby('ticker', group_keys=False)

# SMA 20/50/100
for w in [20, 50, 100]:
    temp_df[f'SMA_{w}'] = g['Close'].transform(lambda s, w=w: sma(s, w))

# EMA 20/50/100
for w in [20, 50, 100]:
    temp_df[f'EMA_{w}'] = g['Close'].transform(lambda s, w=w: ema(s, w))

# RSI 14
temp_df['RSI'] = g['Close'].transform(rsi_wilder)

# MACD (12,26,9)
macd_df = g['Close'].apply(macd)
temp_df = temp_df.join(macd_df)

# Stock Splits
temp_df["forward_split"] = yfinance_df["Stock Splits"].apply(
    lambda x: x if x > 1 else 0
)
temp_df["reverse_split"] = yfinance_df["Stock Splits"].apply(
    lambda x: 1 / x if 0 < x < 1 else 0
)

SP500_all_stock_data = temp_df


SP500_all_stock_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,is_in_sp500,daily_log_return,daily_volume_return,monthly_log_return,monthly_log_return_future,monthly_var_3,daily_vol_21,SMA_20,SMA_50,SMA_100,EMA_20,EMA_50,EMA_100,RSI,MACD_Line,MACD_Signal,MACD_Hist,forward_split,reverse_split
0,2012-01-03,22.776930,23.507775,22.713378,23.183662,4156394,0.0,0.0,A,0,NaN,NaN,NaN,0.181041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,2012-01-04,22.973936,23.107394,22.618046,22.999355,4651845,0.0,0.0,A,0,-0.007982,-0.007982,NaN,0.224083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2,2012-01-05,22.802342,23.717486,22.700660,23.514122,6842651,0.0,0.0,A,0,0.022135,0.022135,NaN,0.188160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3,2012-01-06,23.571322,23.870016,23.393379,23.768333,4711400,0.0,0.0,A,0,0.010753,0.010753,NaN,0.176063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4,2012-01-09,23.908145,24.416559,23.812817,24.391138,4429563,0.0,0.0,A,0,0.025866,0.025866,NaN,0.158459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846056,2021-01-25,156.198734,156.804086,154.152080,155.026474,1443800,0.0,0.0,ZTS,0,-0.012258,-0.012258,-0.001484,NaN,0.000085,0.010760,157.476836,156.053404,155.762710,156.839598,156.179267,153.560021,45.446989,0.129223,0.474895,-0.345672,0.0,0.0
846057,2021-01-26,155.036089,155.314735,152.355259,153.460266,1982000,0.0,0.0,ZTS,0,-0.010154,-0.010154,0.000672,NaN,0.000101,0.010647,157.440262,155.957611,155.744579,156.517756,156.072640,153.558045,41.893553,-0.151293,0.349658,-0.500950,0.0,0.0
846058,2021-01-27,151.807581,152.086226,146.580455,147.156967,2235600,0.0,0.0,ZTS,0,-0.041942,-0.041942,-0.046697,NaN,0.000715,0.013951,157.008414,155.731146,155.631986,155.626253,155.723005,153.431291,31.289987,-0.872173,0.105292,-0.977464,0.0,0.0
846059,2021-01-28,148.261947,154.363450,148.213900,150.404678,1859300,0.0,0.0,ZTS,0,0.021830,0.021830,-0.035205,NaN,0.000611,0.014675,156.703933,155.558315,155.614440,155.128960,155.514444,153.371358,39.751451,-1.167949,-0.149357,-1.018593,0.0,0.0


In [103]:
# for daily prices we use spreads instead of absolute prices
SP500_all_stock_data['high_low_spread'] = (SP500_all_stock_data['High'] - SP500_all_stock_data['Low'])/SP500_all_stock_data['Close']
SP500_all_stock_data['open_close_spread'] = (SP500_all_stock_data['Open'] - SP500_all_stock_data['Close'])/SP500_all_stock_data['Close']


In [104]:
# filter for stocks from 2013-01-01 onwards and before 2020-12-31

SP500_all_stock_data_final = SP500_all_stock_data[
    (SP500_all_stock_data['Date'] >= pd.to_datetime('2013-01-01')) &
    (SP500_all_stock_data['Date'] <= pd.to_datetime('2020-12-31'))
]

SP500_all_stock_data_final

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,is_in_sp500,daily_log_return,daily_volume_return,monthly_log_return,monthly_log_return_future,monthly_var_3,daily_vol_21,SMA_20,SMA_50,SMA_100,EMA_20,EMA_50,EMA_100,RSI,MACD_Line,MACD_Signal,MACD_Hist,forward_split,reverse_split,high_low_spread,open_close_spread
250,2013-01-02,27.067739,27.067739,26.413031,26.881598,8790205,0.0,0.0,A,1,0.022701,0.022701,0.092068,0.078278,0.000440,0.016378,25.679972,24.407813,24.462086,25.703241,24.966327,24.735124,64.959669,0.571168,0.559054,0.012114,0.0,0.0,0.024355,0.006924
251,2013-01-03,26.920109,27.048485,26.689035,26.977879,5751791,0.0,0.0,A,1,0.003575,0.003575,0.105353,0.051698,0.000357,0.016059,25.808129,24.478157,24.472193,25.824635,25.045212,24.779535,65.650086,0.602698,0.567783,0.034915,0.0,0.0,0.013324,-0.002141
252,2013-01-04,27.048492,27.568409,26.868767,27.510639,6432897,0.0,0.0,A,1,0.019556,0.019556,0.119386,0.048723,0.000187,0.016371,25.945957,24.570554,24.486733,25.985206,25.141895,24.833616,69.259449,0.663032,0.586833,0.076199,0.0,0.0,0.025432,-0.016799
253,2013-01-07,27.343756,27.472131,27.202544,27.311663,3589505,0.0,0.0,A,1,-0.007259,-0.007259,0.098323,0.057980,0.000115,0.016493,26.084721,24.655770,24.501007,26.111536,25.226984,24.882686,66.451035,0.686874,0.606841,0.080033,0.0,0.0,0.009871,0.001175
254,2013-01-08,27.260301,27.459282,27.022808,27.093414,3896925,0.0,0.0,A,1,-0.008023,-0.008023,0.099134,0.064228,0.000142,0.016461,26.228901,24.742896,24.514058,26.205048,25.300177,24.926463,63.413659,0.680316,0.621536,0.058780,0.0,0.0,0.016110,0.006160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846037,2020-12-24,153.424249,155.295040,153.357100,154.191757,417400,0.0,0.0,ZTS,1,0.005428,0.005428,-0.002113,-0.046697,0.000210,0.009240,153.626682,155.672884,154.709177,154.477094,154.663478,151.699305,48.711774,-0.298371,-0.514775,0.216405,0.0,0.0,0.012568,-0.004978
846038,2020-12-28,154.882506,156.216045,153.798400,155.793915,1522400,0.0,0.0,ZTS,1,0.010337,0.010337,0.013640,-0.035205,0.000435,0.009425,153.670815,155.665280,154.755871,154.602505,154.707809,151.780386,52.813333,-0.160576,-0.443935,0.283360,0.0,0.0,0.015518,-0.005850
846039,2020-12-29,156.580664,158.393894,155.803553,156.494308,1188400,0.0,0.0,ZTS,1,0.004486,0.004486,0.010167,-0.054363,0.000069,0.009320,153.802251,155.660954,154.774980,154.782677,154.777868,151.873731,54.525404,0.005085,-0.354131,0.359216,0.0,0.0,0.016552,0.000552
846040,2020-12-30,156.868441,158.106051,156.532666,157.597580,1009000,0.0,0.0,ZTS,1,0.007025,0.007025,0.023966,NaN,0.000052,0.009269,153.941843,155.746915,154.828596,155.050763,154.888445,151.987075,57.162074,0.222828,-0.238739,0.461568,0.0,0.0,0.009984,-0.004627


In [105]:
SP500_all_stock_data_final.isna().sum()

Date                            0
Open                            0
High                            0
Low                             0
Close                           0
Volume                          0
Dividends                       0
Stock Splits                    0
ticker                          0
is_in_sp500                     0
daily_log_return               25
daily_volume_return            25
monthly_log_return            525
monthly_log_return_future    1240
monthly_var_3                 575
daily_vol_21                  525
SMA_20                        475
SMA_50                       1225
SMA_100                      2521
EMA_20                        475
EMA_50                       1225
EMA_100                      2521
RSI                           350
MACD_Line                     625
MACD_Signal                   825
MACD_Hist                     825
forward_split                   0
reverse_split                   0
high_low_spread                 0
open_close_spr

In [106]:
# pruning new stocks with insufficient data to compute the fields
SP500_all_stock_data_final = SP500_all_stock_data_final.dropna()
len(SP500_all_stock_data_final)

999466

In [107]:
SP500_all_stock_data_final.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'ticker', 'is_in_sp500', 'daily_log_return',
       'daily_volume_return', 'monthly_log_return',
       'monthly_log_return_future', 'monthly_var_3', 'daily_vol_21', 'SMA_20',
       'SMA_50', 'SMA_100', 'EMA_20', 'EMA_50', 'EMA_100', 'RSI', 'MACD_Line',
       'MACD_Signal', 'MACD_Hist', 'forward_split', 'reverse_split',
       'high_low_spread', 'open_close_spread'],
      dtype='object')

### Log and power transformation


In [108]:
from scipy.stats import skew
import numpy as np
merged_log = SP500_all_stock_data_final[SP500_all_stock_data_final['ticker'] == 'AAPL']

# Select numeric columns
numeric_cols = merged_log.select_dtypes(include=[np.number]).columns

# Compute skewness
skew_vals = merged_log[numeric_cols].apply(lambda x: skew(x.dropna()))
skew_df = pd.DataFrame({"feature": skew_vals.index, "skew": skew_vals.values})
skew_df = skew_df.sort_values(by="skew", ascending=False)

print(skew_df)

                      feature       skew
5                Stock Splits  34.802556
23              forward_split  34.802556
11              monthly_var_3   4.542362
25            high_low_spread   2.914408
4                      Volume   2.593618
12               daily_vol_21   2.530234
1                        High   1.735462
0                        Open   1.731715
3                       Close   1.723551
2                         Low   1.714323
13                     SMA_20   1.695600
16                     EMA_20   1.693827
14                     SMA_50   1.677175
17                     EMA_50   1.658905
15                    SMA_100   1.596228
18                    EMA_100   1.577765
21                MACD_Signal   1.475931
20                  MACD_Line   1.451852
26          open_close_spread   0.234675
19                        RSI  -0.053482
8         daily_volume_return  -0.482895
7            daily_log_return  -0.482895
9          monthly_log_return  -0.526033
10  monthly_log_

/var/folders/sr/m5w47sf11d525m54586gg3c00000gn/T/ipykernel_64621/666708876.py:9: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew_vals = merged_log[numeric_cols].apply(lambda x: skew(x.dropna()))


In [109]:
# 1) Volume
SP500_all_stock_data_final['volume_log'] = np.log1p(SP500_all_stock_data_final['Volume'])

# 2) Volatility/variance (from log returns), clipping to avoid log(0)
eps = 1e-12
SP500_all_stock_data_final['log_daily_vol_21']  = np.log(SP500_all_stock_data_final['daily_vol_21'].clip(lower=eps))
SP500_all_stock_data_final['log_monthly_var_3'] = np.log(SP500_all_stock_data_final['monthly_var_3'].clip(lower=eps))

# 3) Prices & MAs → ratios (then drop raw) https://anderson-review.ucla.edu/wp-content/uploads/2021/03/Avramov-Kaplanski-Subra_2018_SSRN-id3111334.pdf?
for span in [20, 50, 100]:
    SP500_all_stock_data_final[f'close_sma_ratio_{span}'] = (SP500_all_stock_data_final['Close'] - SP500_all_stock_data_final[f'SMA_{span}']) / SP500_all_stock_data_final[f'SMA_{span}']
    SP500_all_stock_data_final[f'close_ema_ratio_{span}'] = (SP500_all_stock_data_final['Close'] - SP500_all_stock_data_final[f'EMA_{span}'])/ SP500_all_stock_data_final[f'EMA_{span}']


# 4) MACD normalized by price
SP500_all_stock_data_final['macd_line_pct']   = SP500_all_stock_data_final['MACD_Line']   / SP500_all_stock_data_final['Close']
SP500_all_stock_data_final['macd_signal_pct'] = SP500_all_stock_data_final['MACD_Signal'] / SP500_all_stock_data_final['Close']
SP500_all_stock_data_final['macd_hist_pct']   = SP500_all_stock_data_final['MACD_Hist']   / SP500_all_stock_data_final['Close']


# 7) Drop raw columns that cause skew / redundancy
to_drop = [
    'Open','High','Low','Close',
    'SMA_20','SMA_50','SMA_100','EMA_20','EMA_50','EMA_100',
    'Volume','daily_vol_21','monthly_var_3',
    'MACD_Line','MACD_Signal','MACD_Hist','RSI',
    'Stock Splits', 'monthly_log_return'
]
SP500_all_stock_data_final = SP500_all_stock_data_final.drop(columns=[c for c in to_drop if c in SP500_all_stock_data_final.columns], errors='ignore')


/var/folders/sr/m5w47sf11d525m54586gg3c00000gn/T/ipykernel_64621/1619745008.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SP500_all_stock_data_final['volume_log'] = np.log1p(SP500_all_stock_data_final['Volume'])
/var/folders/sr/m5w47sf11d525m54586gg3c00000gn/T/ipykernel_64621/1619745008.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SP500_all_stock_data_final['log_daily_vol_21']  = np.log(SP500_all_stock_data_final['daily_vol_21'].clip(lower=eps))
/var/folders/sr/m5w47sf11d525m54586gg3c00000gn/T/

#### Stationarity checks
we will use ADF test for stationarity checks and eliminate serial correlation through transformations if required

In [110]:
from statsmodels.tsa.stattools import adfuller, kpss
import numpy as np

In [111]:
def check_adf_all_columns(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    results = []

    for col in numeric_cols:
        series = df[col].dropna()

        adf_p = np.nan
        adf_verdict = "error"
        kpss_p = np.nan
        kpss_verdict = "error"
        combined_verdict = "inconclusive"

        # ADF Test
        try:
            p_val = adfuller(series, autolag="AIC")[1]  # uses AIC to determine optimal lags
            adf_p = p_val
            adf_verdict = "stationary" if p_val < 0.05 else "non_stationary"
        except Exception as e:
            adf_p = np.nan
            adf_verdict = f"adf_error: {e}"

        # KPSS Test
        # logic is inverted due to h0 being stationary for KPSS in contrast to ADF 
        try:
            kpss_res = kpss(series, regression="c", nlags="auto")  # c is chosen to test for level stationarity, auto will use newey-west lags
            kpss_p = kpss_res[1]
            kpss_verdict = "stationary" if kpss_p >= 0.05 else "non_stationary"
        except Exception as e:
            kpss_p = np.nan
            kpss_verdict = f"kpss_error: {e}"

        # Combine ADF and KPSS results verdict
        if isinstance(adf_verdict, str) and isinstance(kpss_verdict, str):
            if adf_verdict == "stationary" and kpss_verdict == "stationary":
                combined_verdict = "stationary"
            elif adf_verdict == "non_stationary" and kpss_verdict == "non_stationary":
                combined_verdict = "non_stationary"
            elif adf_verdict.startswith("adf_error") or kpss_verdict.startswith("kpss_error"):
                combined_verdict = "inconclusive"
            else:
                # Disagreement → flag for manual review (trend, breaks, low power, etc.)
                combined_verdict = "ambiguous"

        results.append({
            "column": col,
            # ADF
            "ADF_p": adf_p,
            "ADF_Verdict": adf_verdict,
            # KPSS
            "KPSS_p": kpss_p,
            "KPSS_Verdict": kpss_verdict,
            # Combined
            "Combined_Verdict": combined_verdict
        })

    return pd.DataFrame(results)

In [112]:
# Using only one stock (appl) for ADF because my kernel keeps crashing and its computationally expensive :"
appl_stock_data = SP500_all_stock_data_final[SP500_all_stock_data_final['ticker'] == 'AAPL']
adf_results = check_adf_all_columns(appl_stock_data)
display(adf_results)


/Users/ash/Documents/School/Y4S1/DSE4212/project/DSE4212-Portfolio-Optimiser/.venv/lib/python3.14/site-packages/statsmodels/tsa/stattools.py:2182: RuntimeWarning: invalid value encountered in scalar divide
  s_hat = s1 / s0
/var/folders/sr/m5w47sf11d525m54586gg3c00000gn/T/ipykernel_64621/3292298288.py:26: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_res = kpss(series, regression="c", nlags="auto")  # c is chosen to test for level stationarity, auto will use newey-west lags
/var/folders/sr/m5w47sf11d525m54586gg3c00000gn/T/ipykernel_64621/3292298288.py:26: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_res = kpss(series, regression="c", nlags="auto")  # c is chosen to test for level stationarity, auto will use newey-west lags
/var/folde

,column,ADF_p,ADF_Verdict,KPSS_p,KPSS_Verdict,Combined_Verdict
0,is_in_sp500,NaN,"adf_error: Invalid input, x is constant",NaN,kpss_error: cannot convert float NaN to integer,inconclusive
1,daily_log_return,1.231359e-26,stationary,0.100000,stationary,stationary
2,daily_volume_return,1.231359e-26,stationary,0.100000,stationary,stationary
3,monthly_log_return_future,1.002596e-07,stationary,0.100000,stationary,stationary
4,forward_split,0.000000e+00,stationary,0.100000,stationary,stationary
5,reverse_split,NaN,"adf_error: Invalid input, x is constant",NaN,kpss_error: cannot convert float NaN to integer,inconclusive
6,high_low_spread,2.939295e-08,stationary,0.010000,non_stationary,ambiguous
7,open_close_spread,0.000000e+00,stationary,0.020315,non_stationary,ambiguous
8,volume_log,1.629588e-02,stationary,0.010000,non_stationary,ambiguous
9,log_daily_vol_21,1.667732e-04,stationary,0.010000,non_stationary,ambiguous


In [113]:
# ## to make stationary, we will take first differences of the non-stationary columns

# # Identify columns that are either non-stationary OR ambiguous
# target_cols = adf_results.loc[
#     adf_results['Combined_Verdict'].isin(['non_stationary', 'ambiguous']),
#     'column'
# ]

# # Make sure your data is sorted by ticker and date
# SP500_all_stock_data_final.sort_values(['ticker', 'Date'], inplace=True)

# # Take first differences per ticker
# for col in target_cols:
#     SP500_all_stock_data_final[f'd_{col}'] = (
#         SP500_all_stock_data_final
#         .groupby('ticker')[col]
#         .transform(lambda x: x.diff())
#     )


In [114]:
SP500_all_stock_data_final.isna().sum()

Date                         0
Dividends                    0
ticker                       0
is_in_sp500                  0
daily_log_return             0
daily_volume_return          0
monthly_log_return_future    0
forward_split                0
reverse_split                0
high_low_spread              0
open_close_spread            0
volume_log                   0
log_daily_vol_21             0
log_monthly_var_3            0
close_sma_ratio_20           0
close_ema_ratio_20           0
close_sma_ratio_50           0
close_ema_ratio_50           0
close_sma_ratio_100          0
close_ema_ratio_100          0
macd_line_pct                0
macd_signal_pct              0
macd_hist_pct                0
dtype: int64

#### Covid hold out set 
https://en.wikipedia.org/wiki/2020_stock_market_crash#:~:text=Though%20the%20crash%20began%20on,13%25%20in%20most%20global%20markets.

Covid stock crash happened on: 20 Feb
we will use 2013-01-01 to 2020-02-19 data as our training test set and 2020-02-19 to 2020-12-12 as our holdout set

Purging will happen later using 100 days during time-series CV

In [115]:
# splitting the data appropriately into training, test and covid stress test data 
# Covid start https://en.wikipedia.org/wiki/2020_stock_market_crash#:~:text=Though%20the%20crash%20began%20on,13%25%20in%20most%20global%20markets.
COVID_start_date = '2020-02-20'
training_data =  SP500_all_stock_data_final[SP500_all_stock_data_final['Date'] < pd.to_datetime(COVID_start_date)]
covid_stress_test_data = SP500_all_stock_data_final[SP500_all_stock_data_final['Date'] >= pd.to_datetime(COVID_start_date)]

In [116]:
# checking the newly listed stocks in the training data
first_trade_training = training_data.groupby("ticker")["Date"].min().reset_index()
first_trade_training["Date"] = pd.to_datetime(first_trade_training["Date"])
first_trade_training.columns = ["ticker", "first_trade_date"]
first_trade_training.query("first_trade_date >= '2013-01-01'")

,ticker,first_trade_date
0,A,2013-01-02
1,AAL,2013-01-02
2,AAP,2013-01-02
3,AAPL,2013-01-02
4,ABBV,2013-05-24
...,...,...
501,YUM,2013-01-02
502,ZBH,2013-01-02
503,ZBRA,2013-01-02
504,ZION,2013-01-02


In [117]:
training_data.columns

Index(['Date', 'Dividends', 'ticker', 'is_in_sp500', 'daily_log_return',
       'daily_volume_return', 'monthly_log_return_future', 'forward_split',
       'reverse_split', 'high_low_spread', 'open_close_spread', 'volume_log',
       'log_daily_vol_21', 'log_monthly_var_3', 'close_sma_ratio_20',
       'close_ema_ratio_20', 'close_sma_ratio_50', 'close_ema_ratio_50',
       'close_sma_ratio_100', 'close_ema_ratio_100', 'macd_line_pct',
       'macd_signal_pct', 'macd_hist_pct'],
      dtype='object')

In [118]:
training_data[(training_data['is_in_sp500'] != 1) & (training_data['ticker'] == 'TSLA')]

,Date,Dividends,ticker,is_in_sp500,daily_log_return,daily_volume_return,monthly_log_return_future,forward_split,reverse_split,high_low_spread,open_close_spread,volume_log,log_daily_vol_21,log_monthly_var_3,close_sma_ratio_20,close_ema_ratio_20,close_sma_ratio_50,close_ema_ratio_50,close_sma_ratio_100,close_ema_ratio_100,macd_line_pct,macd_signal_pct,macd_hist_pct
1136601,2013-01-02,0.0,TSLA,0,0.043051,0.043051,0.079869,0.0,0.0,0.020928,-0.010181,16.701540,-3.955618,-7.424715,0.033540,0.042919,0.099687,0.082627,0.152865,0.117732,0.013565,0.017196,-0.003631
1136602,2013-01-03,0.0,TSLA,0,-0.016826,-0.016826,0.081966,0.0,0.0,0.020132,0.011792,16.225155,-3.967343,-7.550809,0.015005,0.023031,0.076831,0.061874,0.131534,0.096930,0.014235,0.016837,-0.002603
1136603,2013-01-04,0.0,TSLA,0,-0.010699,-0.010699,0.102945,0.0,0.0,0.025582,0.011628,16.129036,-3.991409,-7.737808,0.003193,0.010975,0.060995,0.048495,0.117678,0.083428,0.013714,0.016357,-0.002644
1136604,2013-01-07,0.0,TSLA,0,-0.001746,-0.001746,0.131600,0.0,0.0,0.026208,0.013396,15.707116,-3.993954,-9.829874,0.000801,0.008327,0.054921,0.044754,0.114136,0.079795,0.012913,0.015691,-0.002779
1136605,2013-01-08,0.0,TSLA,0,-0.019407,-0.019407,0.158890,0.0,0.0,0.041271,0.024347,16.773541,-3.968173,-8.614431,-0.017732,-0.010010,0.031294,0.023684,0.091833,0.057805,0.010794,0.014958,-0.004164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138391,2020-02-12,0.0,TSLA,0,-0.009198,-0.009198,-0.339111,0.0,0.0,0.034381,0.013789,19.010341,-2.653949,-5.754160,0.179965,0.147900,0.519677,0.406117,0.928027,0.715725,0.108249,0.100515,0.007734
1138392,2020-02-13,0.0,TSLA,0,0.046734,0.046734,-0.591368,0.0,0.0,0.103234,-0.077313,19.792723,-2.650151,-5.842039,0.209859,0.180026,0.563332,0.446536,0.992070,0.769851,0.104049,0.097551,0.006498
1138393,2020-02-14,0.0,TSLA,0,-0.004950,-0.004950,-0.620399,0.0,0.0,0.034336,-0.016012,19.276820,-2.663316,-7.214100,0.178478,0.155037,0.528050,0.415011,0.955163,0.734964,0.103562,0.099140,0.004422
1138394,2020-02-18,0.0,TSLA,0,0.070421,0.070421,-0.865583,0.0,0.0,0.032199,-0.019571,19.319726,-2.656932,-5.705473,0.232869,0.211692,0.607280,0.488009,1.065745,0.830320,0.100112,0.093941,0.006171


In [119]:
covid_stress_test_data[covid_stress_test_data['ticker'] == 'TSLA']

,Date,Dividends,ticker,is_in_sp500,daily_log_return,daily_volume_return,monthly_log_return_future,forward_split,reverse_split,high_low_spread,open_close_spread,volume_log,log_daily_vol_21,log_monthly_var_3,close_sma_ratio_20,close_ema_ratio_20,close_sma_ratio_50,close_ema_ratio_50,close_sma_ratio_100,close_ema_ratio_100,macd_line_pct,macd_signal_pct,macd_hist_pct
1138396,2020-02-20,0.0,TSLA,0,-0.019826,-0.019826,-0.743715,0.0,0.0,0.057882,0.013942,19.393441,-2.653011,-6.098006,0.229936,0.207866,0.614504,0.493150,1.096547,0.849815,0.104967,0.094755,0.010212
1138397,2020-02-21,0.0,TSLA,0,0.001766,0.001766,-0.729793,0.0,0.0,0.036193,0.006637,19.184855,-2.652300,-5.453178,0.205020,0.186276,0.585400,0.467262,1.068484,0.822302,0.105753,0.096821,0.008932
1138398,2020-02-24,0.0,TSLA,0,-0.077524,-0.077524,-0.501423,0.0,0.0,0.049533,0.006249,19.244343,-2.607774,-5.576369,0.095429,0.087656,0.442519,0.339023,0.888480,0.663754,0.107367,0.105174,0.002193
1138399,2020-02-25,0.0,TSLA,0,-0.041482,-0.041482,-0.394320,0.0,0.0,0.087010,0.061369,19.373718,-2.596238,-5.713641,0.034480,0.039160,0.362815,0.270433,0.789244,0.577527,0.101617,0.108026,-0.006409
1138400,2020-02-26,0.0,TSLA,0,-0.026745,-0.026745,-0.388355,0.0,0.0,0.047766,0.004751,19.168707,-2.591567,-7.639328,-0.006434,0.010606,0.308168,0.225520,0.721399,0.519768,0.092733,0.107310,-0.014577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138609,2020-12-22,0.0,TSLA,1,-0.014758,-0.014758,0.318831,0.0,0.0,0.055674,0.011962,18.859890,-3.156899,-5.903821,0.041299,0.056853,0.265666,0.202717,0.423875,0.401902,0.069645,0.072434,-0.002788
1138610,2020-12-23,0.0,TSLA,1,0.008769,0.008769,0.312659,0.0,0.0,0.044785,-0.021332,18.415859,-3.196049,-6.982075,0.042789,0.059486,0.266832,0.203245,0.425355,0.402743,0.065637,0.070568,-0.004931
1138611,2020-12-24,0.0,TSLA,1,0.024150,0.024150,0.266840,0.0,0.0,0.037913,-0.028378,18.043756,-3.236273,-6.123037,0.060763,0.076628,0.287673,0.221511,0.448550,0.424702,0.062644,0.067636,-0.004992
1138612,2020-12-28,0.0,TSLA,1,0.002897,0.002897,0.230131,0.0,0.0,0.031039,0.016303,18.388527,-3.246407,-6.752216,0.057238,0.071613,0.280702,0.214338,0.441185,0.416804,0.060867,0.066126,-0.005259
